In [13]:
import sqlite3
import pandas as pd

db_path = 'D:\\code\\uom_explore\\database\\voc_lab.db'

conn = sqlite3.connect(db_path)

# Define the specific conditions
experiment_batch = 'exp_efficiency_test_async_1'
experiment_id = '02_17_16s1c0r0'

# Write the query to select specific columns based on conditions
query = f"""
SELECT channel_id, heater_setting, timestamp, sensor_value
FROM ExperimentData
WHERE experiment_batch = ?
AND experiment_id = ?
"""

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(query, conn, params=(experiment_batch, experiment_id))

# Close the database connection
conn.close()

grouped = df.groupby('heater_setting', as_index=False, group_keys=False)

def normalize_timstamp(group):
    group['timestamp'] = group['timestamp'] - group['timestamp'].iloc[0]
    return group

df_ts = grouped.apply(normalize_timstamp).reset_index(drop=True)
grouped_ts = df_ts.groupby('heater_setting')

print (grouped_ts.head(5))

     channel_id  heater_setting  timestamp  sensor_value
0             0             140          0        7052.0
1             0             150          0        6948.0
2             0             152          0        6453.0
3             0             155          0        5875.0
4             0             157          0        5369.0
..          ...             ...        ...           ...
200           0             240        181        5970.0
201           0             242        181        6105.0
202           0             245        181        6105.0
203           0             247        181        6237.0
204           0             250        180        6479.0

[205 rows x 4 columns]


C:\Users\gavin\AppData\Local\Temp\ipykernel_20444\1889706086.py:32: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [10]:
# use plotly to explore the data interactively
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from scipy.signal import butter, filtfilt


# Filter requirements.
order = 6
fs = 30.0  # sample rate, Hz
cutoff = 3.66  # desired cutoff frequency of the filter, Hz
window_size = 30
group_channel = 'sensor_value'

# apply low pass filter to each group
def apply_filter(group, order, fs, cutoff, group_channel, window_size):
    # Get the filter coefficients
    b, a = butter(order, cutoff / (fs / 2), btype='low', analog=False)
    group['ADC_filtered'] = filtfilt(b, a, group[group_channel])
    group['ADC_filtered'] = group['ADC_filtered'].rolling(window_size, center=True).median()
    return group


# Apply the filter to each group
df_normalized = grouped_ts.apply(apply_filter, order=order, fs=fs, cutoff=cutoff, group_channel=group_channel, window_size=window_size).reset_index(drop=True)
input_voltage = 3.3
RL_2 = 10000 # 10kOhm

# convert to resistance where R_laod is R2 = 10kOhm input_voltage = 3.3V and 1_bit = 0.125mV (3.3V/2^14)
df_normalized['Vo'] = df_normalized['ADC_filtered']*0.000125
df_normalized['Resistance'] = (RL_2/df_normalized['Vo'])*((input_voltage/df_normalized['Vo'])-1)


# Define the figure without subplots as we are plotting only one channel
fig = go.Figure()

# The channel to plot
channel = 'Resistance' 
# 'ADC_filtered'

# Plot the channel
for setting in df_normalized['heater_setting'].unique():
    # Filter the DataFrame for each setting
    df_filtered = df_normalized[df_normalized['heater_setting'] == setting]
    fig.add_trace(
        go.Scatter(
            x=df_filtered['timestamp'],
            y=df_filtered[channel],
            mode='lines',
            name=f'Setting {setting}',
            # Optionally, you can uncomment and adjust the line color settings if needed
            # line=dict(color=colors[setting % len(colors)])  # Loop through colors cyclically
        )
    )

# Set titles and axes labels
fig.update_xaxes(title_text='Timestamp')
fig.update_yaxes(title_text=channel)
fig.update_layout(height=500, width=800, title_text=f'{channel} vs. Timestamp', showlegend=True)

# Display figure
fig.show()

C:\Users\gavin\AppData\Local\Temp\ipykernel_20444\3899351365.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [14]:
# show the number of unique heater_settings
print (df_normalized['heater_setting'].unique())

[140 150 152 155 157 160 162 165 167 170 172 175 177 180 182 185 187 190
 192 195 197 200 202 205 210 212 215 217 220 222 225 227 230 232 235 237
 240 242 245 247 250]


# Transform each experiment into a single feature 

|feature 1|feature 2|feature 3...| ground truth (class)
- each feature is the ratio of a temperature
- each row is a dataset
- each file contain 41 settings 

In [21]:
# from the database retrieve the all experiment_id, heater_setting, sensor_value, channel_id

db_path = 'D:\\code\\uom_explore\\database\\voc_lab.db'

conn = sqlite3.connect(db_path)

# Write the query to select specific columns based on conditions
query = f"""
SELECT experiment_id, heater_setting, timestamp, sensor_value, channel_id
FROM ExperimentData
"""

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(query, conn)
df.head()

,experiment_id,heater_setting,timestamp,sensor_value,channel_id
0,02_17_16s1c0r0,140,62654,7052.0,0
1,02_17_16s1c0r0,150,62656,6948.0,0
2,02_17_16s1c0r0,152,62657,6453.0,0
3,02_17_16s1c0r0,155,62658,5875.0,0
4,02_17_16s1c0r0,157,62659,5369.0,0


In [49]:

# group df by experiment_id and heater_setting
grouped = df.groupby(['experiment_id', 'heater_setting'], as_index=False, group_keys=False)

# normalise
def normalize_timstamp(group):
    group['timestamp'] = group['timestamp'] - group['timestamp'].iloc[0]
    return group

df_ts = grouped.apply(normalize_timstamp).reset_index(drop=True)
grouped_ts = df_ts.groupby(['experiment_id','heater_setting'], as_index=False, group_keys=False)

# Filter requirements.
order = 6
fs = 30.0  # sample rate, Hz
cutoff = 3.66  # desired cutoff frequency of the filter, Hz
window_size = 30
target_channel = 'sensor_value'

# apply low pass filter to each group
def apply_filter(group, order, fs, cutoff, target_channel, window_size):
  # Get the filter coefficients
#   b, a = butter(order, cutoff / (fs / 2), btype='low', analog=False)
#   group['filtered'] = filtfilt(b, a, group[group_channel])
  group['filtered'] = group[target_channel].rolling(window_size, center=False).median()
  return group


# within each experiment_id group, apply the filter to each heater_setting group
# df_filtered = grouped_ts.apply(apply_filter, order=order, fs=fs, cutoff=cutoff, target_channel=group_channel, window_size=window_size).reset_index(drop=True)

filtered_data = []
for name, group in grouped_ts:
  filtered_group = group.copy()  # Avoid modifying original data
  filtered_group['filtered'] = group['sensor_value'].rolling(window_size, center=False).mean()
  filtered_data.append(filtered_group)

df_filtered = pd.concat(filtered_data, ignore_index=True)


# convert to resistance where R_laod is R2 = 10kOhm input_voltage = 3.3V and 1_bit = 0.125mV (3.3V/2^14)
input_voltage = 3.3
RL_2 = 10000 # 10kOhm

# convert to resistance where R_laod is R2 = 10kOhm input_voltage = 3.3V and 1_bit = 0.125mV (3.3V/2^14)
df_normalized['Vo'] = df_filtered['sensor_value']*0.000125
df_normalized['Resistance'] = (RL_2/df_normalized['Vo'])*((input_voltage/df_normalized['Vo'])-1)


C:\Users\gavin\AppData\Local\Temp\ipykernel_20444\3331595845.py:9: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## Retrieve a specific group

In [54]:
import plotly.express as px

experiment_id_value = "02_17_16s1c0r0"	# Example value for experiment_id
heater_setting_value = 140  # Example value for heater setting

# Use a tuple to get the specific group
group = grouped_ts.get_group((experiment_id_value, heater_setting_value))

# Now you can plot this group
fig = px.line(group, x='timestamp', y='sensor_value', title=f'Experiment {experiment_id_value} - Heater Setting {heater_setting_value}')
fig.show()

## Retrieve with index

In [56]:
# plot one specific group with plotly
import plotly.express as px

# Group by 'experiment_id' and 'heater_setting'
grouped_ts = df_ts.groupby(['experiment_id', 'heater_setting'], as_index=False, group_keys=False)

# Convert the groups to a list
groups = list(grouped_ts.groups.keys())

# Use the index to get the specific group
index = 1  # Replace with the desired index
experiment_id_value, heater_setting_value = groups[index]

# Fetch the specific group
group = grouped_ts.get_group((experiment_id_value, heater_setting_value))

# Now you can plot this group
fig = px.line(group, x='timestamp', y='sensor_value', title=f'Experiment {experiment_id_value} - Heater Setting {heater_setting_value}')
fig.show()
